In [1]:
import sqlite3
import zipfile

import pandas as pd

from datetime import datetime
from tools import criaChaveAcordaoPrincipal

## Conhecer a base de dados dos acórdãos do TCU

A base de dados dos acórdãos do TCU foi disponibilizada para download público na plataforma Kaggle. Os dados dizem respeito aos acórdãos proferidos pelo TCU entre os anos de 1992 até 30/08/2019.
Acesse o endereço <https://www.kaggle.com/ferraz/acordaos-tcu> e baixe o arquivo no link Download (4GB) e salve na pasta '/dados'.
github: <https://github.com/netoferraz/acordaos-tcu>

In [3]:
# with zipfile.ZipFile('dados/330881_660826_bundle_archive.zip', 'r') as zip_ref:
#     zip_ref.extractall()

In [6]:
# Cria conexão com a base de datos sqlite
conn = sql ite3.connect('dados/tcu-acordaos.db')

In [5]:
# Cria o dataframe df_acordaos
df_acordaos = pd.read_sql_query('SELECT * from acordaos', conn)

In [7]:
print(df_acordaos.shape, '\n')

# Mostra as colunas da base
print(df_acordaos.columns, '\n')

print(df_acordaos.info(), '\n')

(298942, 20) 

Index(['id', 'urn', 'ano_acordao', 'numero_acordao', 'relator', 'processo',
       'tipo_processo', 'data_sessao', 'numero_ata',
       'interessado_reponsavel_recorrente', 'entidade', 'representante_mp',
       'unidade_tecnica', 'repr_legal', 'assunto', 'sumario', 'acordao',
       'quorum', 'relatorio', 'voto'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   id                                 298942 non-null  int64 
 1   urn                                298942 non-null  object
 2   ano_acordao                        298942 non-null  int64 
 3   numero_acordao                     298123 non-null  object
 4   relator                            298470 non-null  object
 5   processo                           298562 non-null  object
 6   tipo_processo   

In [8]:
# Transfora em string as colunas ano_acordao
df_acordaos['ano_acordao'] = df_acordaos['ano_acordao'].map(str)

In [9]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['representante_mp', 'assunto', 'unidade_tecnica', 'repr_legal', 'quorum',
                               'interessado_reponsavel_recorrente'], axis=1)

In [10]:
df_acordaos.rename(str.upper, axis='columns', inplace=True)
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              298942 non-null  int64 
 1   URN             298942 non-null  object
 2   ANO_ACORDAO     298942 non-null  object
 3   NUMERO_ACORDAO  298123 non-null  object
 4   RELATOR         298470 non-null  object
 5   PROCESSO        298562 non-null  object
 6   TIPO_PROCESSO   298470 non-null  object
 7   DATA_SESSAO     298562 non-null  object
 8   NUMERO_ATA      298470 non-null  object
 9   ENTIDADE        297777 non-null  object
 10  SUMARIO         269290 non-null  object
 11  ACORDAO         298447 non-null  object
 12  RELATORIO       269199 non-null  object
 13  VOTO            269199 non-null  object
dtypes: int64(1), object(13)
memory usage: 31.9+ MB


In [11]:
print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.PROCESSO.isnull()].shape[0])
print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.TIPO_PROCESSO == 'SIGILOSO')].shape[0])
print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.ACORDAO == 'SIGILOSO') | (df_acordaos.ACORDAO == 'None')].shape[0])
print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.VOTO == 'SIGILOSO')  | (df_acordaos.VOTO == 'None')].shape[0])
print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.ENTIDADE == 'SIGILOSO')  | (df_acordaos.ENTIDADE == 'None')].shape[0])
print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.SUMARIO == 'Número de Acórdão não utilizado'].shape[0])

Qtde de acordãos sem número de processo:  380
Qtde de acordãos com tipo de processo SIGILOSO:  5201
Qtde de acordãos com acordao SIGILOSO:  1787
Qtde de acordãos com voto SIGILOSO:  2597
Qtde de acordãos com entidade SIGILOSO:  5542
Qtde de acordãos com Número de Acórdão não utilizado:  92


In [12]:
df_acordaos[(df_acordaos['ENTIDADE'] == 'SIGILOSO') | (df_acordaos['TIPO_PROCESSO'].str.upper == 'SIGILOGO')
       | (df_acordaos['SUMARIO'].str.upper == 'SIGILOGO')
       | (df_acordaos['ACORDAO'].str.upper == 'SIGILOGO')
       | (df_acordaos['RELATORIO'].str.upper == 'SIGILOGO')
       | (df_acordaos['ENTIDADE'].str.upper == 'SIGILOGO')
       | (df_acordaos['VOTO'].str.upper == 'SIGILOGO')
        | (df_acordaos.SUMARIO == 'Número de Acórdão não utilizado')].shape[0]

5630

In [13]:
df_acordaos = df_acordaos[df_acordaos.PROCESSO.notnull()]
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[df_acordaos['TIPO_PROCESSO'] =='SIGILOSO'])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['ACORDAO'] == 'SIGILOSO') | (df_acordaos['ACORDAO'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['VOTO'] == 'SIGILOSO') | (df_acordaos['VOTO'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['ENTIDADE'] == 'SIGILOSO') | (df_acordaos['ENTIDADE'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['SUMARIO'] == 'Número de Acórdão não utilizado')])
print(df_acordaos.shape[0])

298562
293361
293087
292277
292207
292115


In [14]:
print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.PROCESSO.isnull()].shape[0])
print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.TIPO_PROCESSO == 'SIGILOSO')].shape[0])
print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.ACORDAO == 'SIGILOSO') | (df_acordaos.ACORDAO == 'None')].shape[0])
print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.VOTO == 'SIGILOSO')  | (df_acordaos.VOTO == 'None')].shape[0])
print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.ENTIDADE == 'SIGILOSO')  | (df_acordaos.ENTIDADE == 'None')].shape[0])
print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.SUMARIO == 'Número de Acórdão não utilizado'].shape[0])

Qtde de acordãos sem número de processo:  0
Qtde de acordãos com tipo de processo SIGILOSO:  0
Qtde de acordãos com acordao SIGILOSO:  0
Qtde de acordãos com voto SIGILOSO:  0
Qtde de acordãos com entidade SIGILOSO:  0
Qtde de acordãos com Número de Acórdão não utilizado:  0


In [15]:
df_acordaos[(df_acordaos['ENTIDADE'] == 'SIGILOSO') | (df_acordaos['TIPO_PROCESSO'].str.upper == 'SIGILOGO')
       | (df_acordaos['SUMARIO'].str.upper == 'SIGILOGO')
       | (df_acordaos['ACORDAO'].str.upper == 'SIGILOGO')
       | (df_acordaos['RELATORIO'].str.upper == 'SIGILOGO')
       | (df_acordaos['ENTIDADE'].str.upper == 'SIGILOGO')
       | (df_acordaos['VOTO'].str.upper == 'SIGILOGO')
        | (df_acordaos.SUMARIO == 'Número de Acórdão não utilizado')].shape[0]

0

In [16]:
df_acordaos[df_acordaos.duplicated(['NUMERO_ACORDAO', 'PROCESSO', 'DATA_SESSAO'])]

,ID,URN,ANO_ACORDAO,NUMERO_ACORDAO,RELATOR,PROCESSO,TIPO_PROCESSO,DATA_SESSAO,NUMERO_ATA,ENTIDADE,SUMARIO,ACORDAO,RELATORIO,VOTO
294848,294849,NA,2019,2011,AROLDO CEDRAZ,017.162/2007-1,Tomada de contas especial instaurada em razão ...,28/08/2019,33/2019-Plenário,Associação dos Irrigantes da Barragem de Terra...,TOMADA DE CONTAS ESPECIAL. NÃO COMPROVAÇÃO DA ...,"VISTOS, relatados e discutidos estes autos de ...","Inicio este Relatório transcrevendo, com algun...","Conforme consignado no Relatório precedente, t..."
295941,295942,NA,2019,2011,AROLDO CEDRAZ,017.162/2007-1,Tomada de contas especial instaurada em razão ...,28/08/2019,33/2019-Plenário,Associação dos Irrigantes da Barragem de Terra...,TOMADA DE CONTAS ESPECIAL. NÃO COMPROVAÇÃO DA ...,"VISTOS, relatados e discutidos estes autos de ...","Inicio este Relatório transcrevendo, com algun...","Conforme consignado no Relatório precedente, t..."


In [17]:
df_acordaos = df_acordaos.drop(295941)

In [18]:
df_acordaos[df_acordaos.URN == 'NA'].shape[0]

49041

In [19]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292114 entries, 0 to 298941
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              292114 non-null  int64 
 1   URN             292114 non-null  object
 2   ANO_ACORDAO     292114 non-null  object
 3   NUMERO_ACORDAO  292114 non-null  object
 4   RELATOR         292114 non-null  object
 5   PROCESSO        292114 non-null  object
 6   TIPO_PROCESSO   292114 non-null  object
 7   DATA_SESSAO     292114 non-null  object
 8   NUMERO_ATA      292114 non-null  object
 9   ENTIDADE        291421 non-null  object
 10  SUMARIO         263189 non-null  object
 11  ACORDAO         292091 non-null  object
 12  RELATORIO       263190 non-null  object
 13  VOTO            263190 non-null  object
dtypes: int64(1), object(13)
memory usage: 33.4+ MB


In [20]:
start_time = datetime.now()

df_acordaos['ID'] = df_acordaos.apply(criaChaveAcordaoPrincipal, axis=1)

print('Duration: {}'.format(datetime.now() - start_time))

Duration: 0:00:09.241384


In [21]:
# Salva o arquivo em um csv compactado reduzindo o arquivo de 4GB para ~900MB
start_time = datetime.now()

df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False, compression='gzip', encoding='utf-8')

print('Duration: {}'.format(datetime.now() - start_time))

KeyboardInterrupt: 